# Putting it all together!

Taking the folium work from Eboni and putting it into Flask

In [1]:
from flask import Flask
from folium.plugins import HeatMap, HeatMapWithTime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium

In [ ]:
full_year = pd.read_csv('./modis-terra-day-full-year (1).csv', index_col = 0)
full_year['brightness'] = full_year['brightness'] / full_year['brightness'].sum()
full_year['acq_date'] = full_year['acq_date'].sort_values(ascending=True)

In [ ]:
# created a list that groups latitude, longitude and brightness (as the weight) by acquisition date
data = []
for _, d in full_year.groupby('acq_date'):
    data.append([[row['latitude'], row['longitude'], row['brightness']] for _, row in d.iterrows()])
# created a time index for heatmap
time_index = [k[0] for k in full_year.groupby('acq_date')]

In [ ]:
app = Flask(__name__)

@app.route('/')
def index():
    
    ## Fire Data
    full_year = pd.read_csv('./modis-terra-day-full-year (1).csv', index_col = 0)
    full_year['brightness'] = full_year['brightness'] / full_year['brightness'].sum()
    full_year['acq_date'] = full_year['acq_date'].sort_values(ascending=True)
    # created a list that groups latitude, longitude and brightness (as the weight) by acquisition date
    data = []
    for _, d in full_year.groupby('acq_date'):
        data.append([[row['latitude'], row['longitude'], row['brightness']] for _, row in d.iterrows()])
    # created a time index for heatmap
    time_index = [k[0] for k in full_year.groupby('acq_date')]


    ## COVID DATA
    url = 'https://raw.githubusercontent.com/python-visualization/folium/master/examples/data'
    state_geo = f'{url}/us-states.json'
    state_data = pd.read_csv('./covidcast-indicator-combination-confirmed_incidence_num-2020-02-20-to-2020-10-22.csv', index_col=0)
    to_drop = state_data[state_data['geo_value'] == 'PR'].index
    state_data.drop(to_drop, inplace= True)
    state_dict = {state: number for state, number in zip(state_data['geo_value'], range(0, 52))}
    state_data['state_id'] = state_data['geo_value'].map(state_dict)
    state_data['geo_value'] = [i for i in state_data['geo_value']]
    state_data['value'].replace([-16, -46, -140, -8, -613, -1, -28, -31, -26, -398, -41], 0, inplace=True)
    bins = np.linspace(min(state_data['value']),max(state_data['value']), 11)
    

    state_data['color'] = pd.cut(state_data['value'], bins, labels = ['#e1f2f6','#b8d3e3','#b9d2e4','#a1c1db','#b8ccdf','#a4abd0','#9ba2cb','#8e6eb3','#927db8','#9b4395'], include_lowest = False)    
    state_data['color'].replace(to_replace = [np.nan], value = ['#EFF5F6'],inplace=True)
    state_data_color = state_data[['time_value', 'state_id', 'color']]

    state_data_color['state_id'] = state_data_color['state_id'].astype(str)
    state_data_color['time_value'] = pd.to_datetime(state_data_color['time_value'])

    state_data_color['time_value']=(state_data_color['time_value'].astype(int)// 10**9).astype('U10')
    covid_dict={}
    for i in state_data_color['state_id'].unique():
        covid_dict[i]={}
        for j in state_data_color[state_data_color['state_id']==i].set_index(['state_id']).values:   
            covid_dict[i][j[0]]={'color':j[1],'opacity':0.8}

    state_geojson=gpd.read_file(state_geo)
    state_dict = {state.upper(): number for state, number in zip(state_data['geo_value'], range(0, 52))}
    state_geojson['state_id']=state_geojson['id'].map(state_dict)
    state_geojson.drop(columns=['id', 'name'], inplace=True)

    
    

    start_coords = (39.833333, -98.133333)
    folium_map = folium.Map(width = 750, height = 500,left = '30%', location=start_coords, zoom_start=4)
    hm = HeatMapWithTime(data, index= time_index, name = 'Heat Map', auto_play=True, min_opacity=0.4, radius=5)
    hm.add_to(folium_map)

    folium.LayerControl().add_to(folium_map) 
    return f"""
    <html>
       
        <body>
            <h1> <center> Fires and COVID </center> </h1>
            <p1> <center> or some shit iunno </center> </p1>
            <p2> <center> i just work here </center> </p2>
            <p3>{folium_map._repr_html_()}</p3>
            <h2> TEST </h2>
        </body>
    </html>
    """

In [ ]:
if __name__ == '__main__':
    app.run(debug=True)